In [1]:
! pip install -qU google-cloud-aiplatform langchain chromadb pypdf transformers gradio

Restart the Runtime for Vertex AI. The below code crashes the runtime so restart manually!

In [1]:
from google.cloud import aiplatform
aiplatform.init(
    # your Google Cloud Project ID or number
    # environment default used is not set
    project='ibm-keras',
    # the Vertex AI region you will use
    # defaults to us-central1
    location='us-central1',
)

Import the libs

In [2]:
import gradio as gr
import vertexai
from google.cloud import aiplatform
from langchain.llms import VertexAI
from langchain.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
PROJECT_ID = "ibm-keras"
REGION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

In [4]:
def summerise_large_pdf(fileUrl):
  url = fileUrl
  loader = PyPDFLoader(url)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
  texts = text_splitter.split_documents(documents)
  chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
  return chain.run(texts)

In [ ]:
def summerise_doc(file_url):
    return summerise_large_pdf(file_url)

sec_file_summeriser_app = gr.Interface(
    fn=summerise_doc,
    inputs=gr.Textbox(lines=3, placeholder="Please Enter the URL of the SEC Filling Here !"),
    outputs="text",
)
sec_file_summeriser_app.launch(share=True,debug=True)